In [65]:
import pandas as pd
import numpy as np
import tensorflow as tf


df = pd.read_csv('winequality-red.csv', sep = ';')
mean = df['quality'].median()
print(mean)

datasets = []
labels = []
inputs = []

for i in range(3):
    split = df.sample(frac = 1/(3-i), random_state = 1)
    df.drop(split.index, axis = 'index', inplace = True)
    datasets.append(split)

for df in datasets:
    labels.append(df.filter(['quality'], axis = 1))
    inputs.append(df.drop(labels = 'quality', axis = 1))
    
datasets = []

for i in range(3):
    inp = tf.convert_to_tensor(inputs[i], dtype = tf.float32)
    lab = tf.convert_to_tensor(labels[i], dtype = tf.float32)
    data = tf.data.Dataset.from_tensor_slices((inp, lab))
    datasets.append(data)

    
train_ds = datasets[0]
test_ds = datasets[1]
valid_ds = datasets[2]

for d in datasets:
    print(d)

6.0
<TensorSliceDataset shapes: ((11,), (1,)), types: (tf.float32, tf.float32)>
<TensorSliceDataset shapes: ((11,), (1,)), types: (tf.float32, tf.float32)>
<TensorSliceDataset shapes: ((11,), (1,)), types: (tf.float32, tf.float32)>


In [60]:
import tensorflow_datasets as tfds


def make_binary(dim, label):
    label = tf.cond(
    mean <= label,
    lambda: 1,
    lambda: 0)
    return dim, label

def preprocess_data(dataset):
    dataset = dataset.map(make_binary, num_parallel_calls=4)
    dataset = dataset.shuffle(128)
    dataset = dataset.batch(32)
    dataset = dataset.prefetch(16)
    return dataset

train_ds = test_ds.apply(preprocess_data)
test_ds = valid_ds.apply(preprocess_data)
print(tf.constant(1))


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
tf.Tensor(1, shape=(), dtype=int32)


In [61]:
class MyModel(tf.keras.Model):
    
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(256, activation=tf.nn.sigmoid)
        self.dense2 = tf.keras.layers.Dense(256, activation=tf.nn.sigmoid)
        self.dense3 = tf.keras.layers.Dense(10, activation=tf.nn.sigmoid)
        
    @tf.function
    def call(self, inputs):
        print(inputs)
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        return x
        
class SimpleDense(tf.keras.layers.Layer):
    
    def __init__(self, units=256):
        super(SimpleDense, self).__init__()
        self.units = units
        self.acitvation = tf.nn.softmax
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.units,), initializer='random_normal', trainable=True)
        
    def call(self, inputs): 
        x = tf.matmul(inputs, self.w) + self.b
        x = self.activation(x)
        return x



In [62]:
def train_step(model, input, target, loss_function, optimizer):
    with tf.GradientTape() as tape:
        prediction = model(input)
        loss = loss_function(target, prediction)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def test(model, test_data, loss_function):
    
    test_accuracy_aggregator = []
    test_loss_aggregator = []
    
    for (input, target) in test_data:
        prediction = model(input)
        sample_test_loss = loss_function(target, prediction)
        sample_test_accuracy = np.argmax(target, axis=1) == np.argmax(prediction, axis=1)
        sample_test_accuracy = np.mean(sample_test_accuracy)
        test_loss_aggregator.append(sample_test_loss.numpy())
        test_accuracy_aggregator.append(np.mean(sample_test_accuracy))
        
    test_loss = tf.reduce_mean(test_loss_aggregator)
    test_accuracy = tf.reduce_mean(test_accuracy_aggregator)
    
    return test_loss, test_accuracy



In [64]:
tf.keras.backend.clear_session()
train_dataset = train_ds.take(500)
test_dataset = test_ds.take(100)

num_epochs = 10
learning_rate = 0.1

model = MyModel()

cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy()

optimizer = tf.keras.optimizers.SGD(learning_rate)

train_losses = []
test_losses = []
test_accuracies = []

test_loss, test_accuracy = test(model, train_dataset, cross_entropy_loss)
test_losses.append(test_loss)
test_accuracies.append(test_accuracy)

train_loss, _ = test(model, train_dataset, cross_entropy_loss)
train_losses.append(train_loss)

for epoch in range(num_epochs):
    print(f'Epoch: {str(epoch)} starting with accuracy {test_accuracies[-1]}')

    epoch_loss_agg = []
    for input,target in train_dataset:
        train_loss = train_step(model, input, target, cross_entropy_loss, optimizer)
        epoch_loss_agg.append(train_loss)
    
    train_losses.append(tf.reduce_mean(epoch_loss_agg))

    test_loss, test_accuracy = test(model, test_dataset, cross_entropy_loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)
    

Tensor("inputs:0", shape=(32, 11), dtype=float32)
Tensor("inputs:0", shape=(32, 11), dtype=float32)


ValueError: Shapes (32,) and (32, 10) are incompatible

In [ ]:
plt.figure()
line1, = plt.plot(train_losses)
line2, = plt.plot(test_losses)
line3, = plt.plot(test_accuracies)
plt.xlabel("Training steps")
plt.ylabel("Loss/Accuracy")
plt.legend((line1,line2, line3),("training","test", "test accuracy"))
plt.show()